Classifier idea and partial implementation was taken from https://github.com/akapoorx00/Fruit-Classifier-PyTorch

In [3]:
!git clone --recursive https://github.com/Horea94/Fruit-Images-Dataset

Cloning into 'Fruit-Images-Dataset'...
remote: Enumerating objects: 377165, done.
remote: Total 377165 (delta 0), reused 0 (delta 0), pack-reused 377165
Receiving objects: 100% (377165/377165), 2.06 GiB | 15.49 MiB/s, done.
Resolving deltas: 100% (1160/1160), done.
Checking out files: 100% (82231/82231), done.


In [24]:
import os

def extract_data(fruits, fr, to):
    print(f'Extracting fr: {fr} -> to {to}')
    for subdir in os.listdir(fr):
        os.makedirs(to, exist_ok=True)
        if subdir in fruits:
            os.system(f'cp -r {fr}/{subdir} {to}/{subdir}')
            print(subdir)


fruits = ['Apple', 'Banana', 'Cocos', 'Lemon', 'Orange']

extract_data(fruits, 'Fruit-Images-Dataset/Training', 'data/Training')
extract_data(fruits, 'Fruit-Images-Dataset/Test', 'data/Test')

Extracting fr: Fruit-Images-Dataset/Training -> to data/Training
Apple
Lemon
Cocos
Orange
Banana
Extracting fr: Fruit-Images-Dataset/Test -> to data/Test
Apple
Lemon
Cocos
Orange
Banana


In [0]:
import random
import torchvision
import torch
import pickle
from torch import nn
from torch.autograd import Variable
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from PIL import Image

In [0]:
transforms_train = T.Compose([T.ToTensor(),
                              T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])
image_data_train = ImageFolder("./data/Training",transform=transforms_train)
image_data_test = ImageFolder("./data/Test",transform=transforms_train)

random.shuffle(image_data_train.samples)
random.shuffle(image_data_test.samples)

In [0]:
classes_idx = image_data_train.class_to_idx
id2class = dict({(v, k) for (k, v) in classes_idx.items()})
classes = len(image_data_train.classes)
len_train_data = len(image_data_train)
len_test_data = len(image_data_test)

In [0]:
train_loader = DataLoader(dataset=image_data_train, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=image_data_test, batch_size=32, shuffle=True)

In [0]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=5, stride=1), 
    nn.ReLU(),
    nn.MaxPool2d(2), 
    nn.Conv2d(64, 64, kernel_size=7, stride=1),
    nn.ReLU(),
    nn.MaxPool2d(3),
    nn.Conv2d(64, 64, kernel_size=7),
    nn.ReLU(),
    nn.MaxPool2d(5),
    nn.Flatten(),
    nn.Linear(64, 100),
    nn.ReLU(),
    nn.Linear(100, classes)
)

In [30]:
device = torch.device("cuda")
model = model.to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
torch.cuda.is_available()

True

In [0]:
# Train the model
def train(epochs):
    model.train()
    losses = []
    for epoch in range(1, epochs+1):
        print ("epoch #", epoch)
        current_loss = 0.0
        for feature, label in train_loader:
            x = Variable(feature.to(device), requires_grad=False).float()
            x = x.to(device)
            y = Variable(label.to(device), requires_grad=False).long()
            y = y.to(device)
            optimizer.zero_grad() 
            y_pred = model(x) 
            correct = y_pred.max(1)[1].eq(y).sum()
            loss = criterion(y_pred, y) 
            print ("loss: ", loss.item())
            current_loss+=loss.item()
            loss.backward() 
            optimizer.step() 
        losses.append(current_loss) 
    return losses

In [0]:
train(100)

In [32]:
model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten()
  (10): Linear(in_features=64, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=5, bias=True)
)

In [33]:
for b, l in train_loader:
    t = b.to(device)
    print(t.size())
    print(model[0](t.to(device)).size())
    print(model[:2](t.to(device)).size())
    print(model[:3](t.to(device)).size())
    print(model[:4](t.to(device)).size())
    print(model[:5](t.to(device)).size())
    print(model[:6](t.to(device)).size())
    print(model[:7](t.to(device)).size())
    print(model[:8](t.to(device)).size())
    print(model[:9](t.to(device)).size())
    print(model[:10](t.to(device)).size())
    print(model[:11](t.to(device)).size())
    print(model[:12](t.to(device)).size())
    print(model[:13](t.to(device)).size())
    break

torch.Size([32, 3, 100, 100])
torch.Size([32, 64, 96, 96])
torch.Size([32, 64, 96, 96])
torch.Size([32, 64, 48, 48])
torch.Size([32, 64, 42, 42])
torch.Size([32, 64, 42, 42])
torch.Size([32, 64, 14, 14])
torch.Size([32, 64, 8, 8])
torch.Size([32, 64, 8, 8])
torch.Size([32, 64, 1, 1])
torch.Size([32, 64])
torch.Size([32, 100])
torch.Size([32, 100])
torch.Size([32, 5])


In [19]:
classes_idx

{'Apple': 0, 'Banana': 1, 'Cocos': 2, 'Lemon': 3, 'Orange': 4}

In [0]:
with open('id2class.pkl', 'wb') as f:
    pickle.dump(id2class, f)

In [0]:
state_dict = model.state_dict()
for key in state_dict.keys():
    state_dict[key] = state_dict[key].to(torch.device('cpu'))

torch.save(state_dict, 'fruit_classifier.pth')

In [0]:
class FruitClassifier(nn.Module):

    def __init__(self, classes, id2class):
        super(FruitClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1), 
            nn.ReLU(),
            nn.MaxPool2d(2), 
            nn.Conv2d(64, 64, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(3),
            nn.Conv2d(64, 64, kernel_size=7),
            nn.ReLU(),
            nn.MaxPool2d(5),
            nn.Flatten(),
            nn.Linear(64, 100),
            nn.ReLU(),
            nn.Linear(100, classes)
        )
        self.id2class = id2class
    
    def forward(self, x):
        return self.model(x)

    def load_weights(self, path):
        self.model.load_state_dict(torch.load(path))

    def predict(self, x):
        return self.id2class[self.model(x).max(1)[1].cpu().item()]


def load_image(filename, device='cpu'):
    transform = T.Compose([T.ToTensor(), 
                           T.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])])
    img = Image.open(filename)
    img_t = transform(img)
    return img_t.unsqueeze(0).to(device)

In [0]:
with open('id2class.pkl', 'rb') as fd:
    id2class = pickle.load(fd)

clf = FruitClassifier(5, id2class)
clf.load_weights('fruit_classifier.pth')